In [ ]:
import os 
os.chdir("/content/drive/MyDrive/Colab Notebooks/Amit_project")

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten, MaxPooling2D, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import joblib

In [1]:
import keras
keras.__version__

'2.4.3'

In [ ]:
data_path = 'CK+48'

In [ ]:
baseModel = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)


for layer in baseModel.layers:
  layer.trainable = False

headModel = baseModel.output
headModel = MaxPooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(7, activation='softmax')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   rotation_range=15,
                                   horizontal_flip = True)


training_set = train_datagen.flow_from_directory(data_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical',
                                                 shuffle=True)



Found 981 images belonging to 7 classes.


In [ ]:
model.fit_generator(training_set,
                              epochs=20,
                             steps_per_epoch = len(training_set))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
62/62 [==============================] - 13s 196ms/step - loss: 1.8022 - accuracy: 0.3335
Epoch 2/20
62/62 [==============================] - 12s 192ms/step - loss: 1.1035 - accuracy: 0.6182
Epoch 3/20
62/62 [==============================] - 12s 195ms/step - loss: 0.9033 - accuracy: 0.6844
Epoch 4/20
62/62 [==============================] - 12s 192ms/step - loss: 0.7801 - accuracy: 0.7417
Epoch 5/20
62/62 [==============================] - 12s 192ms/step - loss: 0.7235 - accuracy: 0.7545
Epoch 6/20
62/62 [==============================] - 12s 194ms/step - loss: 0.5701 - accuracy: 0.7998
Epoch 7/20
62/62 [==============================] - 12s 192ms/step - loss: 0.4722 - accuracy: 0.8412
Epoch 8/20
62/62 [==============================] - 12s 193ms/step - loss: 0.4712 - accuracy: 0.8434
Epoch 9/20
62/62 [==============================] - 12s 190ms/step - loss: 0.4456 - accuracy: 0.8517
Epoch 10/20
62/62 [==============================] - 12s 192ms/step - loss: 0.3356 - accura

# Saving Model

In [ ]:
model_json = model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("emotion_model.h5")